In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tabulate
import jupyter as jp

## Exercise 1: Series

In [ ]:
# 1. create a series of integers for 1st january 2010 - 31 December 2020
indexes = pd.date_range('2010-01-01', '2020-12-31')
dates = pd.Series(range(len(indexes)), index=indexes)
dates

In [ ]:
# 2. compute 7 days moving average without using a for loop
dates.rolling(window=7).mean()

## Exercise 2: Financial data

In [ ]:
# 0. Load the data - setting the index to date and converting it to Datetimeindex
df = pd.read_csv("data/AAPL.csv", header='infer', delimiter=',', index_col='Date', parse_dates=True)
df.head()

### 0. preprocessing the data

In [ ]:
# have a first look of the data
df.describe()

In [ ]:
# check the types of the data: see if there are some conversion to perform
df.dtypes

In [ ]:
# check for missing values
df.isnull().sum()

In [ ]:
# drop the null values
df.dropna(inplace=True)
df.isnull().sum()

In [ ]:
print(df.head().to_markdown())

In [ ]:
# 1. generate a candlestick chart based on the provided Apple stock data.
import plotly.graph_objects as go
fig = go.Figure(
    data=[
        go.Candlestick(
            x=df.index,
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close']
        )
    ]
)
fig.update_layout(
    height=400
)
fig.show()

In [ ]:
# 2. Aggregate the data to last business of each month
aggregation_funcs = {
    'Open': 'mean',       # Average price for Open
    'Close': 'mean',      # Average price for Close
    'Volume': 'sum',       # Sum of the Volumes
    'High': 'max',        # Max of the Highs
    'Low': 'min',         # Min of the Lows
    'Adj Close': 'mean',  # Average price for Adjusted Close
    
}

# Resample on the last business day ('BM') and apply the aggregation functions
result = df.resample('BM').agg(aggregation_funcs)
result.head()

In [ ]:
# 3. Compute daily returns based on the Open price without using a for loop.
result = (df['Open'] - df['Open'].shift(1)) / df['Open'].shift(1)

print(result)

## Exercise 3: Multi asset returns

In [76]:
business_dates = pd.bdate_range('2021-01-01', '2021-12-31')

#generate tickers
tickers = ['AAPL', 'FB', 'GE', 'AMZN', 'DAI']

#create indexs
index = pd.MultiIndex.from_product([business_dates, tickers], names=['Date', 'Ticker'])

# create DFs
market_data = pd.DataFrame(index=index,
                        data=np.random.randn(len(index), 1),
                        columns=['Price'])


In [ ]:
# 1. compute the daily returns (return(d) = (price(d)-price(d-1))/price(d-1)) for all the companies and returns a DataFrame as
market_data.pivot_table(values="Price", index="Date", columns="Ticker").pct_change()
market_data.head()

# Exercise 4: Backtest

In [57]:
df = pd.read_csv("data/AAPL.csv", header='infer', delimiter=',', index_col='Date', parse_dates=True)

In [ ]:
df.info()

In [ ]:
# 1. drop the missing values and compute the daily future return on the adjusted close price
df.dropna(inplace=True)
returns = (df['Adj Close'].shift(-1) - df['Adj Close']) / df['Adj Close']
returns

##### 2. Random bool series

* The information is this series should be interpreted this way:

    * On the 2010-01-01 I receive 1 before the market closes meaning that, if I trust the signal, the close price of day d+1 will increase. I should buy the stock before the market closes.
    * On the 2010-01-02 I receive 0 before the market closes meaning that,, if I trust the signal, the close price of day d+1 will not increase. I should not buy the stock.


In [ ]:
random_booleans = np.random.randint(0,2,len(df.index))
bool_serie = pd.Series(random_booleans, index=df.index,name="Signal")
bool_serie.head()

##### 3. Signal backtesting

In [ ]:
st = df.merge(bool_serie,left_index=True,right_index=True)
# Calculate daily return: (Close_d+1 - Close_d) / Close_d
st['Next_Close'] = st['Close'].shift(-1)
st['Daily_Return'] = (st['Next_Close'] - st['Close']) / st['Close']
st['PnL'] = st['Signal'] * st['Daily_Return'].fillna(0)

strategy_return = st['PnL']
print(strategy_return)

##### 4. strategy processing

In [62]:
total_earned = st['PnL'].sum()
total_invested = st['Signal'].sum()

# Calculate the return of the strategy
strategy_return = (total_earned - total_invested) / total_invested if total_invested != 0 else 0

In [ ]:
st['Signal_Q5'] = 1  # Always buy
st['PnL_Q5'] = st['Signal_Q5'] * st['Daily_Return'].fillna(0)

total_pnl_q3 = st['PnL'].sum()
total_pnl_q5 = st['PnL_Q5'].sum()

print(f"Total PnL for Q3 strategy: {total_pnl_q3:.4f}")
print(f"Total PnL for Q5 strategy: {total_pnl_q5:.4f}")

plt.figure(figsize=(10, 6))
plt.plot(st.index, st['PnL_Q5'], label='Q5: Always Buy', marker='o', linestyle='--')
plt.plot(st.index, st['PnL'], label='Q3: Signal Based', marker='x', linestyle='-')
plt.title('Daily PnL for Q5 (Always Buy) vs Q3 (Signal Based)')
plt.xlabel('Date')
plt.ylabel('Daily PnL')
plt.legend()
plt.grid(True)
plt.show()